In [1]:
import numpy as np
import pandas as pd
from scipy.signal import stft
import matplotlib.pyplot as plt

In [2]:
def loadData(data_set = None):

    if (data_set == None):
        return None
    else:
        data = []
        fptr = open(data_set, "r")
        lines = fptr.readlines()
        for line in lines:
            line = line.strip().split()
            for i in range(len(line)):
                line[i] = float(line[i])
            # print(line)
            data.append(line)
        
        return np.array(data)

In [5]:
def giveFreqCoeff(data_set):
    
    x = data_set[:, 0]
    y = data_set[:, 1]
    z = data_set[:, 2]

    fs = 32
    seg_size = 32
    hop_size = seg_size // 2

    f_x, t_x, Zxx_x = stft(x, fs, nperseg=seg_size, noverlap=hop_size, scaling='psd')
    f_y, t_y, Zxx_y = stft(y, fs, nperseg=seg_size, noverlap=hop_size, scaling='psd')
    f_z, t_z, Zxx_z = stft(z, fs, nperseg=seg_size, noverlap=hop_size, scaling='psd')

    # Magnitude of frequency coefficients for X, Y, Z-axis
    freq_coeffs_x = np.abs(Zxx_x)  
    freq_coeffs_y = np.abs(Zxx_y)  
    freq_coeffs_z = np.abs(Zxx_z)  

    freq_coeffs_x = freq_coeffs_x[0]
    freq_coeffs_y = freq_coeffs_y[0]
    freq_coeffs_z = freq_coeffs_z[0]

    return freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, t_x[1] - t_x[0]


In [4]:
def mergeDataset(data_set, fc_x, fc_y, fc_z, time_diff):

    new_data_set = []
    length = int(time_diff * 32)
    # = min(length, len(fc_x))

    # print(len(data_set), len(fc_x))
    k = 0
    flag = 0
    for i in range(0, len(data_set), length):

        for j in range(length):

            if (i + j) > len(data_set) - 1 or k > len(fc_x):
                flag = 1
                break

            temp = []
            temp.append(data_set[i + j][0])
            temp.append(data_set[i + j][1])
            temp.append(data_set[i + j][2])
            temp.append(fc_x[k])
            temp.append(fc_y[k])
            temp.append(fc_z[k])
            temp.append(data_set[i + j][3])

            new_data_set.append(temp)

            # print("somethisng appended")
        
        if flag:
            break

        k += 1

    
    return new_data_set


        

In [20]:
data_set_1 = loadData("3_19/visual_annotion_3_19.txt")
rating_1 = pd.read_csv("3_19/Rating.csv")
rating_1 = rating_1["Rating"]


In [21]:
freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, time_diff = giveFreqCoeff(data_set_1)

In [22]:
new_data_set = mergeDataset(data_set_1, freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, time_diff)

In [23]:
len(data_set_1)

22336

In [24]:
def witeNewData(new_data_set):

    # print("5:12")  
    file = open("visual_freq_psd.txt", "w") 
    for i in range(0, len(new_data_set)):
        for elem in new_data_set[i]:
            file.write(str(elem) + "\t")
        file.write("\n")
        # file.writelines(f"{new_data_set[i]}\t{new_data_set[i]}\t{new_data_set[i]}\t{new_data_set}\t\n")

    file.close()

witeNewData(new_data_set)

In [25]:
len(new_data_set)

22336

In [26]:
x = data_set_1[:, 0]
y = data_set_1[:, 1]
z = data_set_1[:, 2]

fs = 32
seg_size = 32
hop_size = seg_size // 2

f_x, t_x, Zxx_x = stft(x, fs, nperseg=seg_size, noverlap=hop_size)
f_y, t_y, Zxx_y = stft(y, fs, nperseg=seg_size, noverlap=hop_size)
f_z, t_z, Zxx_z = stft(z, fs, nperseg=seg_size, noverlap=hop_size)

In [ ]:
fig = plt.figure(figsize=(9, 5))
spec = plt.pcolormesh(t_x, f_x, np.abs(Zxx_x))
cbar = plt.colorbar(spec)

plt.title('STFT Spectrogram')
ax = fig.axes[0]
ax.grid(True)
ax.set_title('STFT Magnitude')

ax.set_ylabel('Frequency [Hz]')
ax.set_xlabel('Time [sec]')
fig.show

In [129]:

freq_coeffs_x = np.abs(Zxx_x)
freq_coeffs_x.shape
t_x[:5]

array([0. , 0.5, 1. , 1.5, 2. ])

In [12]:
# Magnitude of frequency coefficients for X, Y, Z-axis
freq_coeffs_x = np.abs(Zxx_x)  
freq_coeffs_y = np.abs(Zxx_y)  
freq_coeffs_z = np.abs(Zxx_z)  

(1397,)

In [172]:
freq_coeffs_z = np.mean(freq_coeffs_z, axis=0)


In [143]:
freq_coeffs_z[:10]

array([2.01334774, 2.6165116 , 2.08104168, 1.54167041, 1.50192039,
       2.74821563, 5.52708719, 3.43208069, 3.34656531, 4.25126541])

In [42]:
print(x, freq_coeffs_x)

[-0.25961899  0.38714278 -0.59462572 ...  0.02012856  2.98186382
 -0.37143739] [[0.12698784 0.0495531  0.10002736 ... 0.02178528 0.14104729 0.00299342]
 [0.11567441 0.04166631 0.09738974 ... 0.16406773 0.22797895 0.05506835]
 [0.08274473 0.0964906  0.11445873 ... 0.14021225 0.14410543 0.07773322]
 ...
 [0.03408262 0.1867545  0.0890008  ... 0.11873832 0.0926552  0.04138756]
 [0.0297205  0.22842615 0.0494959  ... 0.18131414 0.13060486 0.07925463]
 [0.01005903 0.26131465 0.01842983 ... 0.19528973 0.16200093 0.12032045]]


In [45]:
len(x)

3200

In [50]:
len(freq_coeffs_x[50])

65

In [13]:
freq_coeffs_x.shape

(51, 448)